## Exercise 5 | data in f1ex5 | calculate the urban green space area per inhabitant (m2/inhabitant). 

In [1]:
green_shp = '/mnt/d/cwsig/exercicios_2023_24/f1ex5/espacos_verdes.shp'
pop_shp  = '/mnt/d/cwsig/exercicios_2023_24/f1ex5/freg_cmb.shp'

popcol = 'popres11'

In [11]:
import geopandas as gp
from glass.rd.shp import shp_to_obj

In [3]:
green_df = shp_to_obj(green_shp)

pop_df = shp_to_obj(pop_shp)

In [4]:
green_df['fk'] = 1
pop_df['pk'] = 1

join_df = pop_df.merge(green_df, how='inner', left_on='pk', right_on='fk')

In [5]:
pop_df.shape

(19, 7)

In [6]:
green_df.shape

(35, 3)

In [8]:
join_df.columns.values

array(['Freguesia', 'Des_Simpli', 'idconc', 'idfreg', 'popres11',
       'geometry_x', 'pk', 'Id', 'geometry_y', 'fk'], dtype=object)

In [19]:
join_df["geometry_xx"] = gp.GeoSeries(join_df['geometry_x'])
join_df["geometry_yy"] = gp.GeoSeries(join_df['geometry_y'])

s1 = join_df["geometry_xx"]
s2 = join_df["geometry_yy"]

In [25]:
gp.GeoSeries(s1).intersection(gp.GeoSeries(s2))

/home/gisuser/.virtualenvs/genv/lib/python3.10/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


0      POLYGON EMPTY
1      POLYGON EMPTY
2      POLYGON EMPTY
3      POLYGON EMPTY
4      POLYGON EMPTY
           ...      
660    POLYGON EMPTY
661    POLYGON EMPTY
662    POLYGON EMPTY
663    POLYGON EMPTY
664    POLYGON EMPTY
Length: 665, dtype: geometry

In [16]:
idf = join_df.geometry_xx.intersection(join_df.geometry_yy, align=True)

AttributeError: 'Series' object has no attribute 'intersection'

In [22]:
s2

0      POLYGON ((-36300.929 57807.993, -36298.409 577...
1      POLYGON ((-23679.862 58607.914, -23661.887 585...
2      POLYGON ((-22961.245 58706.742, -22961.155 587...
3      POLYGON ((-27245.231 58717.636, -27247.518 586...
4      POLYGON ((-25497.676 59036.658, -25454.631 589...
                             ...                        
660    POLYGON ((-25688.978 61304.550, -25709.338 612...
661    POLYGON ((-25682.573 61214.551, -25687.489 612...
662    POLYGON ((-23635.989 61256.447, -23638.210 612...
663    POLYGON ((-26808.307 61596.778, -26775.361 615...
664    POLYGON ((-20856.116 60298.261, -20856.111 602...
Name: geometry_yy, Length: 665, dtype: geometry